In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.conv_learner import *
# from fastai.dataset import *
from fastai.models.resnet import vgg_resnet50

import json
from glob import glob

In [3]:
torch.backends.cudnn.benchmark=True

## Data

In [4]:
PATH = Path('../data/all')

In [5]:
def show_img(im, figsize=None, ax=None, alpha=None):
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(im, alpha=alpha)
    ax.set_axis_off()
    return ax

In [6]:
VEHICLES=10
ROADS=7
ROAD_LINES=6

In [7]:
TRAIN_DN = 'CameraRGB'
MASKS_DN = 'CameraSeg'
workers=7
random_crop=True
pseudo_label=False
val_folder = 'sample_test_sync'
# val_folder = 'val'
S_PREFIX = '49_19'

In [8]:
from torchvision.datasets.folder import pil_loader
import torchvision.transforms as transforms
import torchvision.transforms.functional as TTF

### Create dataloader

In [9]:
class MatchedFilesDataset(Dataset):
    def __init__(self, fnames, y, tfms, path):
        self.path,self.fnames = path,fnames
        self.open_fn = pil_loader
        self.y=y
        self.open_y_fn = pil_loader
        assert(len(fnames)==len(y))
        
        self.n = self.get_n()
        self.c = self.get_c()
        self.tfms = tfms
        
    def get_x(self, i): return self.open_fn(os.path.join(self.path, self.fnames[i]))
    def get_y(self, i): return self.open_y_fn(os.path.join(self.path, self.y[i]))
    def get_n(self): return len(self.fnames)
    def get_c(self): return 2
    
    def get(self, tfms, x, y):
        for fn in tfms:
            #pdb.set_trace()
            x, y = fn(x, y)
        return (x, y)
    
    def __getitem__(self, idx):
        x,y = self.get_x(idx),self.get_y(idx)
        return self.get(self.tfms, x, y)
    
    def __len__(self): return self.n

    def resize_imgs(self, targ, new_path):
        dest = resize_imgs(self.fnames, targ, self.path, new_path)
        return self.__class__(self.fnames, self.y, self.transform, dest)

In [10]:

# Seems to speed up training by ~2%
class DataPrefetcher():
    def __init__(self, loader, stop_after=None):
        self.loader = loader
        self.dataset = loader.dataset
        self.stream = torch.cuda.Stream()
        self.stop_after = stop_after
        self.next_input = None
        self.next_target = None

    def __len__(self):
        return len(self.loader)
    
    def preload(self):
        try:
            self.next_input, self.next_target = next(self.loaditer)
        except StopIteration:
            self.next_input = None
            self.next_target = None
            return
        with torch.cuda.stream(self.stream):
            self.next_input = self.next_input.cuda(async=True)
            self.next_target = self.next_target.cuda(async=True)

    def __iter__(self):
        count = 0
        self.loaditer = iter(self.loader)
        self.preload()
        while self.next_input is not None:
            torch.cuda.current_stream().wait_stream(self.stream)
            input = self.next_input
            target = self.next_target
            self.preload()
            count += 1
            yield input, target
            if type(self.stop_after) is int and (count > self.stop_after):
                break

In [11]:
def crop_bg_pil(x,y):
    w, h = x.size
    top = int(h/3.75)
    bot = int(h*.9 + h/150)
    pad_right=32-w%32
    if pad_right == 32: pad_right = 0
    return TTF.crop(x, top, 0, bot-top, w+pad_right), TTF.crop(y, top, 0, bot-top, w+pad_right)

In [12]:
class RHF(object):
    def __init__(self, p=0.5): self.p = p
    def __call__(self, x, y):
        if random.random() < self.p:
            return TTF.hflip(x), TTF.hflip(y)
        return x,y

In [13]:
class RR(object):
    def __init__(self, degrees=2): self.degrees = degrees
    def __call__(self, x, y):
        angle = random.uniform(-self.degrees, self.degrees)
        return TTF.rotate(x, angle), TTF.rotate(y, angle)

In [14]:
def tfm_x_wrapper(tfm):
    return lambda x,y: (tfm(x), y)

In [15]:
class RC():
    def __init__(self, targ_sz):
        self.targ_sz = targ_sz

    def __call__(self, x, y):
        rand_w = random.uniform(0, 1)
        rand_h = random.uniform(0, 1)
        w,h = x.size
        t_w,t_h = self.targ_sz
        start_x = np.floor(rand_w*(w-t_w)).astype(int)
        start_y = np.floor(rand_h*(h-t_h)).astype(int)
        return TTF.crop(x, start_y, start_x, t_h, t_w), TTF.crop(y, start_y, start_x, t_h, t_w)

In [16]:
def convert_y_ce(y_img):
    y_new = np.zeros(y_img.shape, dtype=int)
    y_new[y_img==VEHICLES] = 1
    cutoff_y = int(y_new.shape[0]*.875)
    y_new[cutoff_y:,:] = 0

    y_new[y_img==ROADS] = 2
    y_new[y_img==ROAD_LINES] = 2
    return torch.from_numpy(y_new).long()

In [17]:
def convert_y(y_img):
    yr = (y_img==ROADS) | (y_img==ROAD_LINES)
    yc = (y_img==VEHICLES)
    cutoff_y = int(yc.shape[0]*.875)
    yc[cutoff_y:,:] = 0
    rn = ~(yr | yc)
    return torch.from_numpy(np.stack((rn,yc,yr)).astype(int))


def xy_tensor(x,y):
    y_img = np.array(y, np.int32, copy=False)
    return TTF.to_tensor(x), convert_y_ce(y_img[:,:,0])

In [18]:
class RRC(transforms.RandomResizedCrop):
    def __call__(self, x, y):
        i, j, h, w = self.get_params(x, self.scale, self.ratio)
        x = TTF.resized_crop(x, i, j, h, w, self.size, self.interpolation)
        y = TTF.resized_crop(y, i, j, h, w, self.size, self.interpolation)
        return x, y

In [19]:
def torch_loader(f_ext, data_path, bs, size, workers=7, random_crop=False, pseudo_label=False, val_folder=None, val_bs=None):
    # Data loading code
    x_names = np.sort(np.array(glob(str(data_path/f'CameraRGB{f_ext}'/'*.png'))))
    y_names = np.sort(np.array(glob(str(data_path/f'CameraSeg{f_ext}'/'*.png'))))

    x_n = x_names.shape[0]
    val_idxs = list(range(x_n-300, x_n))
    
    if pseudo_label:
        x_names_test = np.sort(np.array(glob(f'../data/pseudo/CameraRGB{f_ext}/*.png')))
        y_names_test = np.sort(np.array(glob(f'../data/pseudo/CameraSeg{f_ext}/*.png')))
        x_names = np.concatenate((x_names, x_names_test))
        x_names = np.concatenate((y_names, y_names_test))
        print(f'Pseudo-Labels: {len(x_names_test)}')
    if val_folder:
        x_names_val = np.sort(np.array(glob(f'../data/{val_folder}/CameraRGB{f_ext}/*.png')))
        y_names_val = np.sort(np.array(glob(f'../data/{val_folder}/CameraSeg{f_ext}/*.png')))
        val_x,val_y = x_names_val, y_names_val
        trn_x,trn_y = x_names, y_names
        print(f'Val Labels:', len(val_x))
    else:
        ((val_x,trn_x),(val_y,trn_y)) = split_by_idx(val_idxs, x_names, y_names)
    print(f'Val x:{len(val_x)}, y:{len(val_y)}')
    print(f'Trn x:{len(trn_x)}, y:{len(trn_y)}')
    print(f'All x:{len(x_names)}')
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    
    train_tfms = [
        crop_bg_pil,
        tfm_x_wrapper(transforms.ColorJitter(.2,.2,.2)),
#         tfm_x_wrapper(Lighting(0.1, __imagenet_pca['eigval'], __imagenet_pca['eigvec'])),
        RR(),
        RHF(),
#         RC((size,size)),
        xy_tensor,
        tfm_x_wrapper(normalize),
    ]
    if random_crop:
        train_tfms.insert(3,RRC(size, scale=(0.4, 1.0)))
    train_dataset = MatchedFilesDataset(trn_x, trn_y, train_tfms, path='')
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=bs, shuffle=True,
        num_workers=workers, pin_memory=True)

    val_tfms = [
        crop_bg_pil,
        xy_tensor,
        tfm_x_wrapper(normalize)
    ]
    val_dataset = MatchedFilesDataset(val_x, val_y, val_tfms, path='')
    if val_bs is None: val_bs = bs
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=val_bs, shuffle=False,
        num_workers=workers, pin_memory=True)

    train_loader = DataPrefetcher(train_loader)
    val_loader = DataPrefetcher(val_loader)
    
    data = ModelData(data_path, train_loader, val_loader)
    return data


In [20]:
def denorm(x):
    x_np = x.cpu().numpy()
    x_np = np.rollaxis(x_np, 0, 3)
    mean=np.array([0.485, 0.456, 0.406])
    std=np.array([0.229, 0.224, 0.225])
    x_np = x_np*std+mean
    return x_np

## U-net (ish)

In [21]:
from torchvision.models import vgg11_bn

In [22]:
def vgg11(pre): return children(vgg11_bn(pre))[0]

In [23]:
model_meta = {
    resnet18:[8,6], resnet34:[8,6], resnet50:[8,6], resnet101:[8,6], resnet152:[8,6],
    vgg11:[0,13], vgg16:[0,22], vgg19:[0,22],
    resnext50:[8,6], resnext101:[8,6], resnext101_64:[8,6],
    wrn:[8,6], inceptionresnet_2:[-2,9], inception_4:[-1,9],
    dn121:[0,7], dn161:[0,7], dn169:[0,7], dn201:[0,7],
}

In [24]:
def get_base(f):
    cut,lr_cut = model_meta[f]
    layers = cut_model(f(True), cut)
    return nn.Sequential(*layers), lr_cut

In [25]:
class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def remove(self): self.hook.remove()

In [26]:
class UnetBlock(nn.Module):
    def __init__(self, up_in, x_in, n_out):
        super().__init__()
        up_out = x_out = n_out//2
        self.x_conv  = nn.Conv2d(x_in,  x_out,  1)
        self.tr_conv = nn.ConvTranspose2d(up_in, up_out, 2, stride=2)
        self.bn = nn.BatchNorm2d(n_out)
        
    def forward(self, up_p, x_p):
        up_p = self.tr_conv(up_p)
        x_p = self.x_conv(x_p)
        cat_p = torch.cat([up_p,x_p], dim=1)
        return self.bn(F.relu(cat_p, inplace=True))

In [27]:
class Unet34Mod(nn.Module):
    def __init__(self, out=3, f=resnet34):
        super().__init__()
        m_base, lr_cut = get_base(f)
        self.rn = m_base
        self.lr_cut = lr_cut
        self.sfs = [SaveFeatures(self.rn[i]) for i in [2,4,5,6]]
        self.up1 = UnetBlock(512,256,256)
        self.up2 = UnetBlock(256,128,256)
        self.up3 = UnetBlock(256,64,128)
        self.up4 = UnetBlock(128,64,64)
        self.up5 = UnetBlock(64,32,32)
        self.up6 = nn.ConvTranspose2d(32, out, 1)
        self.x_skip = nn.Sequential(
            nn.Conv2d(out,32,1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )
        
    def forward(self,x):
        x_skip = self.x_skip(x)
        x = self.rn(x)
        x = self.up1(x, self.sfs[3].features)
        x = self.up2(x, self.sfs[2].features)
        x = self.up3(x, self.sfs[1].features)
        x = self.up4(x, self.sfs[0].features)
        x = self.up5(x, x_skip)
        x = self.up6(x)
        return torch.squeeze(x)
    
    def close(self):
        for sf in self.sfs: sf.remove()

In [28]:
class Unet34(nn.Module):
    def __init__(self, out=3, f=resnet34):
        super().__init__()
        m_base, lr_cut = get_base(f)
        self.rn = m_base
        self.lr_cut = lr_cut
        self.sfs = [SaveFeatures(self.rn[i]) for i in [2,4,5,6]]
        self.up1 = UnetBlock(512,256,256)
        self.up2 = UnetBlock(256,128,256)
        self.up3 = UnetBlock(256,64,256)
        self.up4 = UnetBlock(256,64,256)
        self.up5 = UnetBlock(256,out,16)
        self.up6 = nn.ConvTranspose2d(16, out, 1)
        
    def forward(self,x):
        inp = x
        x = self.rn(x)
        x = self.up1(x, self.sfs[3].features)
        x = self.up2(x, self.sfs[2].features)
        x = self.up3(x, self.sfs[1].features)
        x = self.up4(x, self.sfs[0].features)
        x = self.up5(x, inp)
        x = self.up6(x)
        return torch.squeeze(x)
    
    def close(self):
        for sf in self.sfs: sf.remove()

In [29]:
class UnetModel():
    def __init__(self,model,name='unet'):
        self.model,self.name = model,name

    def get_layer_groups(self, precompute):
        if isinstance(self.model, FP16):
            model = self.model.module
        else:
            model = self.model
        lgs = list(split_by_idxs(children(model.rn), [model.lr_cut]))
        return lgs + [children(model)[1:]]

In [30]:
def carce_f_p_r(pred, targs):
    _,idx = torch.max(pred, 1)
    return fbeta_score(idx==1, targs[:,:,:]==1, beta=2)

In [31]:
def rdce_f(pred, targs):
    _,idx = torch.max(pred, 1)
    f,p,r = fbeta_score(idx==2, targs[:,:,:]==2, beta=0.5)
    return f

In [32]:
def carsig_f_p_r(pred, targs):
    p2 = F.sigmoid(pred)
    return fbeta_score(p2[:,0,:,:], targs[:,0,:,:], beta=2, threshold=0.5)

In [33]:
def rdsig_f(pred, targs):
    p2 = F.sigmoid(pred)
    f,p,r = fbeta_score(p2[:,1,:,:], targs[:,1,:,:], beta=0.5, threshold=0.5)
    return f

In [34]:
def car_f_p_r(pred, targs):
    _,idx = torch.max(pred, 1)
    return fbeta_score(idx==1, targs[:,1,:,:], beta=2)

In [35]:
def rd_f(pred, targs):
    _,idx = torch.max(pred, 1)
    f,p,r = fbeta_score(idx==2, targs[:,2,:,:], beta=0.5)
    return f

In [36]:
def new_acc_sig(pred, targs):
    p2 = F.sigmoid(pred)
    return ((p2>0.5).long() == targs).float().mean()

In [37]:
def new_acc_ce(preds, targs):
    mx,idx = torch.max(preds, 1)
    return (idx == targs).float().mean()

In [38]:
def new_acc(pred, targs):
    _,idx = torch.max(pred, 1)
    _,t_idx = torch.max(targs,1)
    return (idx == t_idx).float().mean()

In [39]:
def dice_coeff_weight(pred, target, weight):
    smooth = 1.
    num,c,h,w = pred.shape
    m1 = pred.view(num, c, -1)  # Flatten
    m2 = target.view(num, c, -1)  # Flatten
    intersection = (m1 * m2)
    w = V(weight.view(1,-1,1))
    i_w = (w*intersection).sum()
    m1_w = (w*m1).sum()
    m2_w = (w*m2).sum()
    return (2. * i_w + smooth) / (m1_w + m2_w + smooth)

def dice_coeff(pred, target):
    smooth = 1.
    num,c,h,w = pred.shape
    m1 = pred.view(num, c, -1)  # Flatten
    m2 = target.view(num, c, -1)  # Flatten
    intersection = (m1 * m2).sum()
    return (2. * intersection + smooth) / (m1.sum() + m2.sum() + smooth)


class SoftDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True, softmax=True):
        super(SoftDiceLoss, self).__init__()
        self.weight = weight
        self.softmax = softmax

    def forward(self, logits, targets):
        probs = F.softmax(logits) if self.softmax else F.sigmoid(logits)
        num = targets.size(0)  # Number of batches
        targets = torch.cat(((targets==0).unsqueeze(1), (targets==1).unsqueeze(1), (targets==2).unsqueeze(1)), dim=1).float()
        if isinstance(logits.data, torch.cuda.HalfTensor):
            targets = targets.half()
        else:
            targets = targets.float()
        if self.weight is not None:
            score = dice_coeff_weight(probs, targets, self.weight)
        else:
            score = dice_coeff(probs, targets)
        score = 1 - score.sum() / num
        return score

In [40]:
def fbeta_score(y_pred, y_true, beta, threshold=None, eps=1e-9):
    beta2 = beta**2

    if threshold:
        y_pred = torch.ge(y_pred.float(), threshold).float()
    else:
        y_pred = y_pred.float()
    y_true = y_true.float()

    true_positive = (y_pred * y_true).sum()
    precision = true_positive/(y_pred.sum()+(eps))
    recall = true_positive/(y_true.sum()+eps)
    
    fb = (precision*recall)/(precision*beta2 + recall + eps)*(1+beta2)
    
    return fb, precision, recall

In [41]:
def lyft_score(pred, target, weight):
    num,c,h,w = pred.shape
    pred = pred.view(num, c, -1)  # Flatten
    target = target.view(num, c, -1)  # Flatten
    intersection = (pred * target)
    int_sum = intersection.sum(dim=-1)
    pred_sum = pred.sum(dim=-1)
    targ_sum = target.sum(dim=-1)
    
    eps = 1e-9
    precision = int_sum / (pred_sum + eps)
    recall = int_sum / (targ_sum + eps)
    beta = V(weight ** 2)
    
    fnum = (1.+beta) * precision * recall
    fden = beta * precision + recall + eps
    
    fscore = fnum / fden
    
#     fb = (precision*recall)/precision*beta + recall + eps
    
    avg_w = torch.cuda.FloatTensor([0,.5,.5])
    favg = V(avg_w) * fscore
#     pdb.set_trace()
    return favg.sum(dim=-1)

class FLoss(nn.Module):
    def __init__(self, weight=torch.cuda.FloatTensor([1,2,0.5]), softmax=True):
        super().__init__()
        self.weight = weight
        self.softmax = softmax

    def forward(self, logits, targets):
        probs = F.softmax(logits) if self.softmax else F.sigmoid(logits)
        num = targets.size(0)  # Number of batches
        targets = torch.cat(((targets==0).unsqueeze(1), (targets==1).unsqueeze(1), (targets==2).unsqueeze(1)), dim=1).float()
        if isinstance(logits.data, torch.cuda.HalfTensor):
            targets = targets.half()
        else:
            targets = targets.float()
            
        score = lyft_score(probs, targets, self.weight)
        score = 1 - score.sum() / num
        return score

In [42]:
def get_learner(md, m_fn=Unet34, weights=[1,200,2], half=False, softmax=True, dice=False):
    out_sz = 3 if softmax else 2
    m = to_gpu(m_fn(out_sz))
    models = UnetModel(m)
    learn = ConvLearner(md, models)
    learn.opt_fn=optim.Adam
    class_weights = torch.cuda.FloatTensor(weights)
    if half:
        class_weights = class_weights.half()
        learn.half()
        
#     if dice: learn.crit=SoftDiceLoss(weight=class_weights, softmax=softmax)
#     else: learn.crit=nn.CrossEntropyLoss(weight=class_weights)
    learn.crit = FLoss(softmax=softmax)
    
    if softmax: learn.metrics = [new_acc_ce, rdce_f, carce_f_p_r]
    else: learn.metrics = [new_acc_sig, rdsig_f, carsig_f_p_r]
    # learn.metrics=[new_acc, rd_f, car_f_p_r]
    
    return learn

### Models

In [ ]:
ext = '-300'
sz=192
bs=64
random_crop=True
md = torch_loader(ext, PATH, bs, sz, workers, random_crop, pseudo_label, val_folder)

In [ ]:
learn = get_learner(md, weights=[1,10,2], softmax=True)
learn.load(f'600urn-19-weights')

In [ ]:
learn.lr_find()
learn.sched.plot()

In [ ]:
lr=1e-2
wd=1e-7

lrs = np.array([lr/200,lr/20,lr])/2

In [ ]:
learn.unfreeze()
learn.set_bn_freeze(learn.model.rn, True)

In [ ]:
learn.fit(lr, 1, wds=wd, cycle_len=30,use_clr=(20,2))

In [ ]:
learn.save(f'300urn-{S_PREFIX}-rc')

In [ ]:
ext = '-300'
sz=192
bs=64
random_crop=False
md = torch_loader(ext, PATH, bs, sz, workers, random_crop, pseudo_label, val_folder)

In [ ]:
lr=1e-3
wd=1e-7

lrs = np.array([lr/50,lr/10,lr])

In [ ]:
learn.load(f'300urn-{S_PREFIX}-rc')

In [ ]:
learn.unfreeze()
learn.set_bn_freeze(learn.model.rn, True)

In [ ]:
learn.fit(lrs/5, 1, wds=wd, cycle_len=20,use_clr=(20,10))

In [ ]:
learn.save(f'300urn-{S_PREFIX}-nocrop')

### 512

In [ ]:
ext = ''
sz=320
bs=32
random_crop=True
md = torch_loader(ext, PATH, bs, sz, workers, random_crop, pseudo_label, val_folder, val_bs=16)

In [ ]:
learn = get_learner(md, m_fn=Unet34Mod, weights=[1,4,1], softmax=True, dice=False)
learn.load(f'300urn-{S_PREFIX}-nocrop')

In [ ]:
lr=1e-3
wd=1e-7

lrs = np.array([lr/200,lr/20,lr])

In [ ]:
learn.unfreeze()
learn.set_bn_freeze(learn.model.rn, True)

In [ ]:
learn.fit(lrs, 1, wds=wd, cycle_len=6,use_clr=(20,2))

In [ ]:
learn.save(f'600urn-{S_PREFIX}-320')

### Try old model again

In [ ]:
ext = ''
sz=384
bs=32
random_crop=True
md = torch_loader(ext, PATH, bs, sz, workers, random_crop, pseudo_label, val_folder, val_bs=16)

In [ ]:
learn = get_learner(md, m_fn=Unet34Mod, weights=[1,4,1], softmax=True, dice=False)
# learn.load(f'600urn-{S_PREFIX}-320')
learn.load(f'600urn-46_wide-384-nocrop-w8-pt3')

In [ ]:
lr=1e-3
wd=1e-7

lrs = np.array([lr/200,lr/20,lr])/2

In [ ]:
learn.unfreeze()
learn.set_bn_freeze(learn.model.rn, True)

In [ ]:
learn.fit(lrs, 1, wds=wd, cycle_len=8,use_clr=(20,2))

In [ ]:
learn.save(f'600urn-{S_PREFIX}-384')

### Nocrop

In [ ]:
ext = ''
sz=384
bs=16
random_crop=False
md = torch_loader(ext, PATH, bs, sz, workers, random_crop, pseudo_label, val_folder, val_bs=16)

In [ ]:
learn = get_learner(md, m_fn=Unet34Mod, weights=[1,4,2], softmax=True)
# learn.load(f'600urn-{S_PREFIX}-384')
learn.load(f'600urn-46_wide-384-nocrop-w8-pt3')

In [ ]:
lr=1e-3
wd=1e-7

lrs = np.array([lr/200,lr/20,lr])/2

In [ ]:
learn.unfreeze()
# learn.bn_freeze(True)
learn.set_bn_freeze(learn.model.rn, True)

In [ ]:
learn.fit(lrs, 1, wds=wd, cycle_len=7,use_clr=(20,2))

In [ ]:
learn.save(f'600urn-{S_PREFIX}-384-nocrop')

In [ ]:
learn.load(f'600urn-{S_PREFIX}-384-nocrop')

In [ ]:
learn.fit(lrs/40, 1, wds=wd, cycle_len=6,use_clr=(20,2))

In [ ]:
learn.save(f'600urn-{S_PREFIX}-384-nocrop-2')

### Train on test

In [43]:
ext = ''
sz=384
bs=16
random_crop=False
pseudo_label=False
md = torch_loader(ext, Path(f'../data/test_sync'), bs, sz, workers, random_crop, pseudo_label, val_folder, val_bs=16)

Val Labels: 400
Val x:400, y:400
Trn x:1000, y:1000
All x:1000


In [44]:
learn = get_learner(md, weights=[1,4,2], softmax=True)
learn.load(f'../../all/models/600urn-19-weights')
# learn.load(f'600urn-19-weights')

In [45]:
lr=1e-4
wd=1e-7

lrs = np.array([lr/200,lr/20,lr])/2

In [46]:
learn.unfreeze()
# learn.bn_freeze(True)
learn.set_bn_freeze(learn.model.rn, True)

In [47]:
learn.fit(lrs, 1, wds=wd, cycle_len=10,use_clr=(20,2))

epoch      trn_loss   val_loss   new_acc_ce rdce_f     carce_f_p_r 
    0      0.145589   0.140568   0.989221   0.992648   0.879309   0.77879    0.909724  
    1      0.146041   0.140217   0.989179   0.992665   0.878673   0.776768   0.9096    
    2      0.147214   0.138435   0.988865   0.992884   0.879978   0.761951   0.916789  
    3      0.147191   0.135986   0.988644   0.993092   0.880492   0.74143    0.925724  
    4      0.143483   0.134043   0.988989   0.993292   0.881569   0.74931    0.924034  
    5      0.141316   0.131587   0.989126   0.99343    0.88248    0.755073   0.923048  
    6      0.140458   0.129971   0.989436   0.993471   0.881575   0.768563   0.916804  
    7      0.137621   0.128197   0.989189   0.993441   0.884149   0.753464   0.926153  
    8      0.136814   0.127147   0.989139   0.993576   0.88489    0.749395   0.92884   
    9      0.13755    0.127155   0.989183   0.993588   0.884994   0.749412   0.928939  



[0.12715522050857545,
 0.9891828393936157,
 0.9935882425308228,
 0.8849935531616211,
 0.7494120323657989,
 0.9289394092559814]

In [48]:
learn.save(f'../../all/models/600urn-{S_PREFIX}-384-test-sync')

In [49]:
learn.fit(lrs*2, 1, wds=wd, cycle_len=10,use_clr=(20,2))

epoch      trn_loss   val_loss   new_acc_ce rdce_f     carce_f_p_r 
    0      0.134497   0.126123   0.989258   0.993564   0.885389   0.751565   0.928666  
    1      0.133935   0.125068   0.989338   0.993729   0.886101   0.7576     0.927235  
    2      0.131956   0.122689   0.9896     0.993699   0.886667   0.768026   0.92401   
    3      0.127743   0.12019    0.989379   0.993864   0.888559   0.755588   0.931495  
    4      0.124045   0.116311   0.989709   0.994036   0.889538   0.772333   0.926364  
    5      0.121281   0.112143   0.989514   0.99413    0.893563   0.759595   0.936756  
    6      0.115918   0.109233   0.989654   0.994186   0.894487   0.770647   0.933615  
    7      0.112806   0.106225   0.989764   0.994164   0.896186   0.772854   0.935147  
    8      0.11171    0.10426    0.989864   0.994234   0.897799   0.774356   0.936681  
    9      0.108841   0.103488   0.989813   0.9942     0.89829    0.776818   0.936318  



[0.10348783969879151,
 0.9898127055168152,
 0.9941997098922729,
 0.8982896041870118,
 0.7768183517456054,
 0.936317777633667]

In [50]:
learn.save(f'../../all/models/600urn-{S_PREFIX}-384-test-sync-2')

In [51]:
learn.load(f'../../all/models/600urn-{S_PREFIX}-384-test-sync-2')

In [52]:
learn.unfreeze()
# learn.bn_freeze(True)
learn.set_bn_freeze(learn.model.rn, True)

In [53]:
learn.fit(lrs*2, 1, wds=wd, cycle_len=20,use_clr=(20,2))

epoch      trn_loss   val_loss   new_acc_ce rdce_f     carce_f_p_r 
    0      0.109571   0.103681   0.989738   0.994312   0.898351   0.774741   0.937158  
    1      0.109245   0.10312    0.989967   0.994205   0.899333   0.779178   0.936713  
    2      0.107333   0.101936   0.98974    0.994269   0.899105   0.779921   0.936093  
    3      0.107632   0.100909   0.98986    0.994381   0.900073   0.779503   0.937624  
    4      0.10554    0.098829   0.989888   0.994313   0.899509   0.785608   0.934864  
    5      0.105629   0.097595   0.989898   0.994384   0.901546   0.782887   0.938289  
    6      0.103488   0.096498   0.99004    0.994336   0.902227   0.793128   0.9353    
    7      0.101206   0.09404    0.989796   0.994431   0.904113   0.77648    0.944302  
    8      0.10019    0.092669   0.989892   0.994464   0.905269   0.778302   0.945178  
    9      0.098402   0.092118   0.989953   0.994522   0.90461    0.790282   0.939601  
    10     0.0948     0.089973   0.989953   0.994467

[0.0825727128982544,
 0.9901059222221374,
 0.9945825791358948,
 0.910714282989502,
 0.7921670985221863,
 0.9474284625053406]

In [55]:
learn.save(f'../../all/models/600urn-{S_PREFIX}-384-test-sync-3')

In [56]:
learn.fit(lrs/2, 1, wds=wd, cycle_len=12,use_clr=(20,2))

epoch      trn_loss   val_loss   new_acc_ce rdce_f     carce_f_p_r 
    0      0.087486   0.082352   0.990123   0.994598   0.910943   0.791662   0.947877  
    1      0.087626   0.082312   0.990226   0.994578   0.911404   0.79091    0.948734  
    2      0.088524   0.082813   0.989965   0.994646   0.910315   0.790322   0.947494  
    3      0.089228   0.08213    0.989996   0.994625   0.911703   0.786285   0.950877  
    4      0.086607   0.082309   0.990047   0.994649   0.910638   0.793551   0.946672  
    5      0.085851   0.081568   0.990181   0.994583   0.911556   0.792005   0.948655  
    6      0.08585    0.082154   0.990063   0.994634   0.909012   0.803239   0.941038  
    7      0.085637   0.081855   0.990155   0.994606   0.910608   0.796023   0.945783  
    8      0.085109   0.0812     0.990103   0.994653   0.911597   0.789399   0.949629  
    9      0.0851     0.081171   0.99039    0.994599   0.910546   0.80068    0.944113  
    10     0.08518    0.081078   0.990097   0.994658

[0.08094205379486084,
 0.9901598238945007,
 0.9946727705001831,
 0.9110101079940796,
 0.7985550284385681,
 0.9454771137237549]

In [57]:
learn.save(f'../../all/models/600urn-{S_PREFIX}-384-test-sync-4')

In [58]:
learn.fit(lrs/20, 1, wds=wd, cycle_len=10,use_clr=(20,2))

epoch      trn_loss   val_loss   new_acc_ce rdce_f     carce_f_p_r 
    0      0.086598   0.081035   0.990039   0.994675   0.912199   0.785793   0.951782  
    1      0.084683   0.081003   0.990256   0.994644   0.911376   0.796076   0.946857  
    2      0.084811   0.080734   0.99027    0.994609   0.911584   0.796699   0.946887  
    3      0.083682   0.081136   0.990165   0.99463    0.910959   0.795896   0.946377  
    4      0.084213   0.080848   0.990212   0.994618   0.911442   0.798561   0.945952  
    5      0.085967   0.080883   0.990261   0.994618   0.911148   0.796144   0.94655   
    6      0.085286   0.080845   0.990237   0.994619   0.911124   0.797462   0.945982  
    7      0.084867   0.080766   0.990303   0.994623   0.911827   0.798674   0.94644   
    8      0.085835   0.080714   0.990123   0.994626   0.911872   0.791557   0.949164  
    9      0.085316   0.080738   0.990152   0.994639   0.911437   0.796116   0.94688   



[0.08073795557022095,
 0.9901516675949097,
 0.9946390604972839,
 0.9114365744590759,
 0.7961157131195068,
 0.9468801212310791]

In [59]:
learn.save(f'../../all/models/600urn-{S_PREFIX}-384-test-sync-5')

### Train on false answers

In [ ]:
ext = ''
sz=384
bs=16
random_crop=False
md = torch_loader(ext, Path(f'../data/test'), bs, sz, workers, random_crop, pseudo_label, val_folder, val_bs=16)

In [ ]:
learn = get_learner(md, m_fn=Unet34Mod, weights=[1,4,2], softmax=True)
learn.load(f'../../all/models/600urn-{S_PREFIX}-384-test-sync-2')

In [ ]:
lr=1e-4
wd=1e-7

lrs = np.array([lr/200,lr/20,lr])/2

In [ ]:
learn.unfreeze()
# learn.bn_freeze(True)
learn.set_bn_freeze(learn.model.rn, True)

In [ ]:
learn.fit(lrs, 1, wds=wd, cycle_len=10,use_clr_beta=(20,2))

In [ ]:
learn.save(f'../../all/models/600urn-{S_PREFIX}-384-test')